# Hur gör man ett HR-diagram i python?

För det första så behöver vi data. I den här labben kommer vi använda oss av data från en satellit som heter GAIA. Om du är intresserad kan du läsa mer om GAIA här(https://www.esa.int/Science_Exploration/Space_Science/Gaia). För att kunna göra sökningar i GAIAs databas måste vi veta vad dom olika parametrarna heter, dessa kan vi hitta här(https://gea.esac.esa.int/archive/) om man klickar på search och sedan advanced kan vi få fram en lista över alla olika datatabeller som finns. Vi kommer använda oss av den som heter gaiadr3.gaia_source_lite. Klicka på det lilla pluset som finns så får ni upp alla columner som finns att söka på där. Leta upp vilka kolumner ni vill ha och skriv ner vad dom heter så ni har det till senare. Nu ska vi börja med lite kod, för att kunna komma åt GAIA databasen från python måste vi importera GAIA modulen från astroquuery, vi behöver också lite saker för att kunna plocka fram koordinater och för att plotta saker så vi importerar detta. 

In [2]:
import numpy as np
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

För att plocka fram koordinaterna för ett objekt kan ni antingen använda SIMBAD som ni gjorde i förra astronomilabben eller så kan ni använda den inbyggda funktionen i astropy som finns nedan.

In [3]:
koordinater = SkyCoord.from_name('M2')#Skriv in namnet på er stjärnhop här och se om ni får ut rätt koordinater

koordinater

<SkyCoord (ICRS): (ra, dec) in deg
    (323.362583, -0.82325)>

Sedan behöver vi skriva en Query till databasen, det är typ som en google sökning fast lite mer avancerad. GAIA databasen kan man söka i eller querya med ett språk som heter ADQL eller astronomical query language, detta är en variant av SQL för den som vet vad det är. Så det vi gör är att vi skriver en lång sträng i python som vi sedan skickar över till databasen som en sökning nedan kan ni se en mall för hur er query bör se ut. 

SELECT *Här skriver ni vilka columner ni vill ha* \
FROM gaiadr3.gaia_source_lite *Här väljer vi vilken tabell vi ska söka i* \
WHERE DISTANCE(323.4, -0.82, ra, dec) < 10 *Här väljer vi kring vilken punkt vi söker och hur stort område vi söker i. Områdets storlek anges i grader* \
AND parallax > 0 *Här filtrerar vi bort alla datapunkter med parallax mindre än 0*\
AND 1000/parallax > 200 *Här kan vi filtrera på avstånd i parsec*\
AND ...

För att se till att den datan ni får är av bra kvalite och med små fel så finns där för varje kolumn ni väljer att hämta en kolumn som heter ..._over_error i den kolumnen har vi mätvärdet dividerat med felet i mätvärdet dessa kan ni använda för att filtrera bort datapunkter med stora mätfel.

För att sedan skicka queryn till GAIA database använder vi nedan kod

In [4]:
query = '''

'''

job = Gaia.launch_job_async(query)#skickar till databasen

results = job.get_results()#Hämtar hem resultaten

results#visar resultaten

INFO: Query finished. [astroquery.utils.tap.core]


abs_mag,bp_rp
,mag
float64,float32
11.113064333628381,2.8172817
6.184602130655341,1.1759014
4.136629142072916,0.84285927
8.236208335192728,1.9636898
4.210872430969744,0.8064375
4.480317294468346,0.8308258
7.093319490893979,1.3679495
0.6956708995773404,1.3487482


För att göra matte med data i en enskild kolumn kan ni skriva kod på formatet nedan. Numpy kan också användas för mer avancerad matte om ni behöver.

In [ ]:
results['kolumn namn'] = results['kolumn namn'] * 3 - 5

När ni har datan på rätt format och i rätt enheter m.m så kan ni börja plotta upp era HR-diagram. För detta kan ni använda pyplot.

In [ ]:
from matplotlib import colors

fig, ax = plt.subplots()#Skapar ett figur och axel objekt 

h = ax.hist2d(temp, magnitud, bins=300, cmin=10, norm=colors.PowerNorm(0.5), zorder=0.5)#Här skapar vi ett 2d histogram för att vi ska kunna se densiteten av stjärnor

ax.scatter(temperatur, magnitud, alpha=0.05, s=1, color='k', zorder=0)#Här plottar vi upp varje stjärna som en punkt
ax.invert_yaxis()

cb = fig.colorbar(h[3], ax=ax, pad=0.02)#ritar en colorbar 

ax.set_xlabel()#Axel titlar
ax.set_ylabel()
cb.set_label()

plt.show()